In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
folder_path = '/content/drive/My Drive/charts'
import os
files = os.listdir(folder_path)
print(files)
!cp -r "/content/drive/My Drive/charts" "/content/charts"

['train_val.csv', 'test', 'train_val']


In [6]:
!pip install tensorflow opencv-python matplotlib scikit-learn pandas

import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#  Data Loading and Preprocessing
train_dir = 'charts/train_val'  # Path to my training images folder
test_dir = 'charts/test'  # Path to my test images folder
csv_file = 'charts/train_val.csv'  # Path to my CSV file

df = pd.read_csv(csv_file, dtype=str)
df['image_index'] = df['image_index'] + '.png'  # Adding file extension because we only know the file name

# Now we split data into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Image dimensions and batch size
img_width, img_height = 128, 128  # I check few images, they have 128 * 128 diminsion
batch_size = 32

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col='image_index',
    y_col='type',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=train_dir,
    x_col='image_index',
    y_col='type',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Model Building Process is start from here
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Add dropout for regularization, Prevents overfitting
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Model Training with our dataset
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

#  Model Evaluation and Prediction
loss, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)





Found 800 validated image filenames belonging to 5 classes.
Found 200 validated image filenames belonging to 5 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.3957 - loss: 1.4151 - val_accuracy: 0.7396 - val_loss: 0.6180
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8750 - val_loss: 0.3781
Epoch 3/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.7961 - loss: 0.4541 - val_accuracy: 0.8906 - val_loss: 0.2869
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.3082
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.8271 - loss: 0.4498 - val_accuracy: 0.8750 - val_loss: 0.2905
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8750 - val_loss: 0.2567
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.8229 - loss: 0.4347 - val_accuracy: 0.8750 - val_loss: 0.2652
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8750 - val_loss: 0.3912
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9255 - loss: 0.2336 - val_accuracy: 0.8646 - val_loss: 0.3586
Epoch 10/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accu

In [7]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2
import matplotlib.pyplot as plt

In [8]:
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator



test_image_paths = [os.path.join(test_dir, filename) for filename in os.listdir(test_dir)
                   if filename.endswith(('.png', '.jpg', '.jpeg'))]

test_images = []
for image_path in test_image_paths:
    img = load_img(image_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = img_array / 255.0
    test_images.append(img_array)

test_images = np.array(test_images)


# Define and instantiate test_datagen
test_datagen = ImageDataGenerator(rescale=1./255)  #  Same preprocessing as we done in training process


test_generator = test_datagen.flow(
    x=test_images,
    y=None,           # Because we have No labels for prediction
    batch_size=batch_size,
    shuffle=False
)

# Predict
predictions = model.predict(test_generator)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 375ms/step


In [9]:
predicted_classes = np.argmax(predictions, axis=1)

# Mapping class indices to labels
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}

predicted_labels = [class_labels[k] for k in predicted_classes]

# Printing out all the predicted labels
print(predicted_labels)



#print image name with predicatd label to ensure everthing work as expected
for i in range(len(test_image_paths)):
    print(f"Image: {os.path.basename(test_image_paths[i])}, Predicted Label: {predicted_labels[i]}")



['pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie', 'pie']
Image: 40.png, Predicted Label: pie
Image: 26.png, Predicted Label: pie
Image: 49.png, Predicted Label: pie
Image: 34.png, Predicted Label: pie
Image: 27.png, Predicted Label: pie
Image: 36.png, Predicted Label: pie
Image: 4.png, Predicted Label: pie
Image: 43.png, Predicted Label: pie
Image: 5.png, Predicted Label: pie
Image: 8.png, Predicted Label: pie
Image: 21.png, Predicted Label: pie
Image: 30.png, Predicted Label: pie
Image: 6.png, Predicted Label: pie
Image: 39.png, Predicted Label: pie
Image: 9.png, Predicted Label: pie
Image: 15.png, Predicted Label: pie
Image: 32.png, Predicted Label: pie
Image: 0.png, Predicted Label: pie
Image: 

In [11]:
# Now we predict image label on my any given image path
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img

def predict_image_label(image_path, model, img_width, img_height):
    img = load_img(image_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)

    # Get the class labels from the training generator
    class_labels = list(train_generator.class_indices.keys())
    predicted_label = class_labels[predicted_class_index]

    return predicted_label

# My Example Image path
image_path = '7.png'
predicted_label = predict_image_label(image_path, model, 128, 128)
print(f"Predicted label: {predicted_label}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
Predicted label: vbar_categorical


In [12]:
# Soon, I will use this model in web application
model.save('model.h5')


In [13]:
pip install tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 75.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


In [14]:
!tensorflowjs_converter --input_format keras model.h5 ./model_js

2024-10-16 17:22:51.131125: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 17:22:51.198093: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 17:22:51.210468: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 17:22:56.117515: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
failed to lookup keras version from the file,
    this is likely a weight only file
